# Quickstart: Modelling a surface using geological observations
This tutorial will demonstrate how to create an implicit surface representation of surfaces from a combination of orientation and location observations. 

Implicit surface representation involves finding an unknown function where $f(x,y,z)$ matches observations of the surface geometry. We generate a scalar field where the scalar value is the distance away from a reference horizon. The reference horizon is arbritary and can either be:

 * a single geological surface where the scalar field would represent the signed distance away from this surface. (above the surface positive and below negative)
 * Where multiple conformable horizons are observed the same scalar field can be used to represent these surfaces and the thickness of the layers is used to determine the relative scalar value for each surface


This tutorial will demonstrate both of these approaches for modelling a number of horizons picked from seismic data sets.

In [1]:
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer 

from LoopStructural.datasets import load_claudius #demo data 

import pandas as pd
import glob
import numpy as np
%matplotlib inline

In [4]:
data, bb = load_claudius()#claudius.get_data()

### Importing data
There are numerous ways to import data into python for LoopStructural. For this example there are three types of files: 

* `Dips.csv` which are the orientation observations X,Y,Z,XOrient,YOrient,ZOrient
* `_Points.csv` which are point observations of the contacts X,Y,Z,Strati
* `_Section.csv` which are section interpetations of the contacts X,Y,Z,Strati

The files can be imported into pandas dataframes. Three dataframes are created one for each of the observations types. 

### GeologicalModel


In [5]:
model = GeologicalModel(bb[0,:],bb[1,:])
model.set_model_data(data)
# model.data
model.create_and_add_conformable_foliation("strati",interpolatortype="FDI",nelements=50000,buffer=0.3)#.2)

In [ ]:
viewer = LavaVuModelViewer(background="white")

# determine the number of unique surfaces in the model from the input data and then calculate isosurfaces for this
unique = np.unique(model.features[0].support.interpolator.get_value_constraints()[:,3])
# print(unique)
viewer.add_isosurface(model.features[0], 
                       slices=unique,  
                       cmap='prism',
                      paint_with=model.features[0])

# viewer.add_isosurface(model.features[1], 
#                        slices=unique,  
#                        colour='blue')
viewer.add_section(model.features[0],
                   axis='x',
                   boundary_points=model.bounding_box, 
                   nsteps=np.array([30,30,30]),
                  cmap='prism')
viewer.add_scalar_field(model.bounding_box,(38,55,30),
                      'box',
                     paint_with=model.features[0],
                     cmap='prism')

# Add the data addgrad/addvalue arguments are optional
viewer.add_data(model.features[0],addgrad=True,addvalue=True, cmap='prism')
viewer.lv.rotate([-85.18760681152344, 42.93233871459961, 0.8641873002052307])
viewer.interactive()# to add an interactive display
# viewer.lv.display()
